In [10]:
from model import *
from data import *

In [11]:
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [12]:
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
gpu_options =tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.5)
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
tf.compat.v1.keras.backend.set_session(sess)
from numpy.random import seed
import random
seed(42)
tf.random.set_seed(42)
random.seed(42)

## Train your Unet with membrane data
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Train with data generator

In [13]:
TIMES = 50
TRAIN_SHAPE = 30
BATCH_SIZE = 2
STEP_PER_EPOCH = TIMES*TRAIN_SHAPE/BATCH_SIZE
EPOCHS = 5
IMG_SIZE = 256
TARGET_SIZE = (IMG_SIZE,IMG_SIZE)

In [14]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(BATCH_SIZE,'data/membrane/train',
                        image_folder='image',
                        mask_folder='label',
                        aug_dict = data_gen_args,
                        flag_multi_class = False,
                        num_class = 2,
                        save_to_dir = None,
                        target_size = TARGET_SIZE,
                        seed = 42)
model = unet()
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=STEP_PER_EPOCH,epochs=EPOCHS,callbacks=[model_checkpoint])

Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
  ...
    to  
  ['...']
Train for 750.0 steps
Epoch 1/5
749/750 [============================>.] - ETA: 0s - loss: 0.2633 - accuracy: 0.8862
Epoch 00001: loss improved from inf to 0.26314, saving model to unet_membrane.hdf5
750/750 [==============================] - 69s 92ms/step - loss: 0.2631 - accuracy: 0.8863
Epoch 2/5
749/750 [============================>.] - ETA: 0s - loss: 0.1785 - accuracy: 0.9222
Epoch 00002: loss improved from 0.26314 to 0.17844, saving model to unet_membrane.hdf5
750/750 [==============================] - 68s 91ms/step - loss: 0.1784 - accuracy: 0.9222
Epoch 3/5
749/750 [============================>.] - ETA: 0s - loss: 0.1523 - accuracy: 0.9330
Epoch 00003: loss improved from 0.17844 to 0.15232, saving model to unet_membrane.hdf5
750/750 [==============================] - 69s 91ms/step - loss: 0.1523 - accuracy: 0.9330
Epoch 4/5
749/750 [============================>.] - ETA: 

### Train with npy file

In [6]:
#imgs_train,imgs_mask_train = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
#model.fit(imgs_train, imgs_mask_train, batch_size=2, nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

### test your model and save predicted results

In [16]:
testGene = testGenerator("data/membrane/test",num_image=30)
model = unet()
model.load_weights("unet_membrane.hdf5")
results = model.predict_generator(testGene,verbose=1)
saveResult("data/membrane/predict",results)

     30/Unknown - 1s 33ms/step

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0